In [ ]:
from typing_extensions import Literal
from langgraph.graph import MessagesState
from typing import TypedDict
import json

# =========================
# 1. State
# =========================
class Router1State(MessagesState):
    question: str
    is_hr_question: Literal["yes", "no"]
    next_step: Literal["rag", "reject"]

# =========================
# 2. Node     Router1 (LLM 기반 판별 - JSON 출력 강제)
# =========================
def router1_node(state: Router1State) -> Router1State:
    prompt = f"""
    당신은 "가이다 플레이 스튜디오(GPS)"의 HR 정책 안내 챗봇입니다.  
    당신은 회사 내부 직원의 질문이 **HR(인사/근무/휴가/복지/장비·보안/출장·비용처리 등)**과 관련된 질문인지 아닌지를 판별하는 것입니다.  

    ## 참고
    - 근무: 하이브리드(재택) 등  
    - 휴가 관련  
    - 장비·보안: 노트북/모니터/장비 보조금, 계정 보안, 도난 절차 등  
    - 복지: 복지포인트, 교육비, 사내 동아리/동호회/모임 지원, 상담 지원, 건강검진 지원,
      카페/스낵바(사무실 내 무료 간식, 커피머신, 음료, 과자, 컵라면 등), 임신/출산/육아 지원 등  
    - 기타: 출장비, 식대, 일비, 급여일, 교통비 등  

    # 금지
    - 개인 HR 데이터(개인 연차 잔여일, 급여·퇴직금 계산 등)  
    
    ### 출력 형식 (반드시 JSON):
    {{
      "is_hr_question": "yes" | "no",
      "next_step": "rag" | "reject"
    }}

    질문: "{state['question']}"
    """

    response = llm.invoke(prompt).content.strip()

    try:
        parsed = json.loads(response)
    except json.JSONDecodeError:
        # LLM이 JSON을 못 주면 기본값으로 fallback
        parsed = {"is_hr_question": "no", "next_step": "reject"}

    return {
        **state,
        "is_hr_question": parsed.get("is_hr_question", "no"),
        "next_step": parsed.get("next_step", "reject"),
    }

# =========================
# 3. Router1 Router
# =========================
def route_after_router1(state: Router1State) -> str:
    return "router2" if state["next_step"] == "rag" else "reject"

# =========================
# 4. Reject Node
# =========================
def reject_node(state: Router1State) -> dict:
    return {
        **state,
        "answer": "❌ 지원하지 않는 질문입니다. HR 관련 문의만 가능합니다."
    }
